In [8]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Algorithm on GPUs?
Consider quicksort and bitsort, which performs better on GPUs?

### Quick sort

In [9]:
render_code("quicksort.cu", show="quicksort")

// quicksort.cu:80-125 (46 lines)
 __global__ static void quicksort(int* values) {
 #define MAX_LEVELS	300

	int pivot, L, R;
	int idx =  threadIdx.x + blockIdx.x * blockDim.x;
	int start[MAX_LEVELS];
	int end[MAX_LEVELS];

	start[idx] = idx;
	end[idx] = N - 1;
	while (idx >= 0) {
		L = start[idx];
		R = end[idx];
		if (L < R) {
			pivot = values[L];
			while (L < R) {
				while (values[R] >= pivot && L < R)
					R--;
				if(L < R)
					values[L++] = values[R];
				while (values[L] < pivot && L < R)
					L++;
				if (L < R)
					values[R--] = values[L];
			}
			values[L] = pivot;
			start[idx + 1] = L + 1;
			end[idx + 1] = end[idx];
			end[idx++] = L;
			if (end[idx] - start[idx] > end[idx - 1] - start[idx - 1]) {
	                        // swap start[idx] and start[idx-1]
        	                int tmp = start[idx];
                	        start[idx] = start[idx - 1];
                        	start[idx - 1] = tmp;

	                        // swap end[idx] and end[idx-1]
        	                tmp = end[idx];
                	        end[idx] = end[idx - 1];
                        	end[idx - 1] = tmp;
	                }

		}
		else
			idx--;
	}
}

In [10]:
render_code("bitonic.cu", show="bitonic_sort")

// bitonic.cu:81-102 (22 lines)
void bitonic_sort(float *values)
{
  float *dev_values;
  size_t size = NUM_VALS * sizeof(float);

  cudaMalloc((void**) &dev_values, size);
  cudaMemcpy(dev_values, values, size, cudaMemcpyHostToDevice);

  dim3 blocks(BLOCKS,1);    /* Number of blocks   */
  dim3 threads(THREADS,1);  /* Number of threads  */

  int j, k;
  /* Major step */
  for (k = 2; k <= NUM_VALS; k <<= 1) {
    /* Minor step */
    for (j=k>>1; j>0; j=j>>1) {
      bitonic_sort_step<<<blocks, threads>>>(dev_values, j, k);
    }
  }
  cudaMemcpy(values, dev_values, size, cudaMemcpyDeviceToHost);
  cudaFree(dev_values);
}

In [14]:
render_code("bitonic.cu", show="bitonic_sort_step")

// bitonic.cu:49-76 (28 lines)
__global__ void bitonic_sort_step(float *dev_values, int j, int k)
{
  unsigned int i, ixj; /* Sorting partners: i and ixj */
  i = threadIdx.x + blockDim.x * blockIdx.x;
  ixj = i^j;

  /* The threads with the lowest ids sort the array. */
  if ((ixj)>i) {
    if ((i&k)==0) {
      /* Sort ascending */
      if (dev_values[i]>dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
    if ((i&k)!=0) {
      /* Sort descending */
      if (dev_values[i]<dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
  }
}

In [11]:
! make bitonic
! ./bitonic

make: 'bitonic' is up to date.
Sorting 262144 numbers
Elapsed time: 0.088s
SORTING SUCCESSFUL


In [22]:
! rm -f quicksort; make quicksort CFLAGS=-DN=262144
!./quicksort

/usr/local/cuda/bin/nvcc -O3 -DN=262144 quicksort.cu -o quicksort
./quicksort starting with 262144 numbers...

------------------------------
Data set distribution: Uniform

Beginning kernel execution...
Elapsed time: 8.513s

Testing results...
SORTING SUCCESSFUL


In [18]:
! rm -f bitonic; make bitonic CFLAGS=-DBLOCKS=512
! ./bitonic

rm -f madd_A madd_B arraySort bitonic quicksort quicksort_cpu
/usr/local/cuda/bin/nvcc -O3 -DBLOCKS=512 bitonic.cu -o bitonic
Sorting 524288 numbers
Elapsed time: 0.091s
SORTING SUCCESSFUL


In [21]:
! rm -f quicksort; make quicksort CFLAGS=-DN=524288
!./quicksort

/usr/local/cuda/bin/nvcc -O3 -DN=524288 quicksort.cu -o quicksort
./quicksort starting with 524288 numbers...

------------------------------
Data set distribution: Uniform

Beginning kernel execution...
Elapsed time: 33.743s

Testing results...
SORTING SUCCESSFUL


In [24]:
! rm -f bitonic; make bitonic CFLAGS=-DBLOCKS=1024
! ./bitonic

/usr/local/cuda/bin/nvcc -O3 -DBLOCKS=1024 bitonic.cu -o bitonic
Sorting 1048576 numbers
Elapsed time: 0.093s
SORTING SUCCESSFUL


In [23]:
! rm -f quicksort; make quicksort CFLAGS=-DN=1048576
!./quicksort

/usr/local/cuda/bin/nvcc -O3 -DN=1048576 quicksort.cu -o quicksort
./quicksort starting with 1048576 numbers...

------------------------------
Data set distribution: Uniform

Beginning kernel execution...
Elapsed time: 134.371s

Testing results...
SORTING SUCCESSFUL


In [17]:
! nvidia-smi -a


==============NVSMI LOG==============

Timestamp                                 : Wed Jul 30 10:44:47 2025
Driver Version                            : 555.42.06
CUDA Version                              : 12.5

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Product Name                          : NVIDIA GeForce RTX 4070 Ti SUPER
    Product Brand                         : GeForce
    Product Architecture                  : Ada Lovelace
    Display Mode                          : Disabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
  